In [1]:
# Importy
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from typing import Callable
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Pobranie danych
heart_disease = fetch_ucirepo(id=45)

# Porzucenie linii z pustymi etykietami oraz odpowiadajacych im wartosci
feature_matrix = heart_disease.data.features.dropna()
labels = heart_disease.data.targets.loc[feature_matrix.index]

# Przetworzenie zbioru wartości przewidywanych do wartości binarnych
y_binary = labels.copy()
y_binary['num'] = y_binary['num'].apply(lambda x: 1 if x != 0 else 0)                      

# Utworznnie zbioru dummy etykiet
x_dummy = pd.get_dummies(feature_matrix, columns=['cp', 'restecg', 'slope','ca','thal'])   

# upewnienie się co do typów wykorzystywanych danych
x_dummy = x_dummy.fillna(0).astype(float)
y_binary = y_binary.astype(float)

# Podział danych na zbiór uczący i testowy
x_train, x_test, y_train, y_test = train_test_split(x_dummy, y_binary, test_size=0.2, random_state=268555)

# Konwersja danych do tablic NumPy przed utworzeniem tensorów PyTorch
X_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)  # wektor kolumnowy
X_test_tensor = torch.tensor(x_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)    # wektor kolumnowy

# # Normalizacja cech trenignowych i testowych
# scaler = StandardScaler()
# x_train_normalised = scaler.fit_transform(x_train)
# x_test_normalised = scaler.transform(x_test)

# # Konwersja danych do wymaganego formatu
# x_train = np.array(x_train).astype(float)
# y_train = np.array(y_train).astype(float)

# x_test = np.array(x_test).astype(float)
# y_test = np.array(y_test).astype(float)
 

In [53]:
# Stworzona klasa Dataset
class HeartDiseaseDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Instancja klasy Dataset
train_dataset = HeartDiseaseDataset(X_train_tensor, y_train_tensor)
test_dataset = HeartDiseaseDataset(X_test_tensor, y_test_tensor)

# Stworzony DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Stworzony model sieci neuronowej (jedna warstwa ukryta, 16 perceptonów)
class HeartDiseaseModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=16):
        super(HeartDiseaseModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Stworzona instancja modelu
input_dim = x_train.shape[1]
model = HeartDiseaseModel(input_dim)

# Funkcja kosztu i optymalizatory
criterion = nn.BCELoss()
optimizers = {
    "SGD": optim.SGD(model.parameters(), lr=0.5),
    "Adam": optim.Adam(model.parameters(), lr=0.5),
    "RMSprop": optim.RMSprop(model.parameters(), lr=0.5)
}

# Trenowanie modelu
def train_model(model, optimizer, num_epochs=100):
    for epoch in range(num_epochs):
        model.train()
        total_cost = 0.0
        for inputs, labels in train_loader:
            labels = labels.squeeze(1)  # Usunięcie dodatkowego wymiaru
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_cost += loss.item()
        
        if(epoch % 10 == 9):
            print(f"Epoch [{epoch+1}/{num_epochs}], Cost: {total_cost / len(train_loader):.4f}")

# Ewaluacja modelu
def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            preds = outputs.round()
            all_preds.extend(preds.numpy().flatten())
            all_labels.extend(labels.numpy().flatten())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    return accuracy, f1, precision

# Trenowanie i ewaluacja z użyciem różnych optymalizatorów
results = {}
for name, optimizer in optimizers.items():
    print(f"\nTraining with {name} optimizer:")
    model = HeartDiseaseModel(input_dim)
    train_model(model, optimizer)
    accuracy, f1, precision = evaluate_model(model, test_loader)
    results[name] = {
        "accuracy": accuracy,
        "f1_score": f1,
        "precision": precision
    }

results


Training with SGD optimizer:
Epoch [10/100], Cost: 1.5469
Epoch [20/100], Cost: 1.5456
Epoch [30/100], Cost: 1.5286
Epoch [40/100], Cost: 1.5409
Epoch [50/100], Cost: 1.5341
Epoch [60/100], Cost: 1.5838
Epoch [70/100], Cost: 1.4978
Epoch [80/100], Cost: 1.5303
Epoch [90/100], Cost: 1.5511
Epoch [100/100], Cost: 1.5510

Training with Adam optimizer:
Epoch [10/100], Cost: 7.9024
Epoch [20/100], Cost: 7.7663
Epoch [30/100], Cost: 7.9708
Epoch [40/100], Cost: 8.0254
Epoch [50/100], Cost: 7.7998
Epoch [60/100], Cost: 7.9581
Epoch [70/100], Cost: 7.7814
Epoch [80/100], Cost: 7.7650
Epoch [90/100], Cost: 7.8265
Epoch [100/100], Cost: 7.8739

Training with RMSprop optimizer:


c:\Users\kkacp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [10/100], Cost: 2.4804
Epoch [20/100], Cost: 2.5052
Epoch [30/100], Cost: 2.4927
Epoch [40/100], Cost: 2.5005
Epoch [50/100], Cost: 2.5046
Epoch [60/100], Cost: 2.4548
Epoch [70/100], Cost: 2.5053
Epoch [80/100], Cost: 2.5352
Epoch [90/100], Cost: 2.4593
Epoch [100/100], Cost: 2.4560


{'SGD': {'accuracy': 0.43333333333333335,
  'f1_score': np.float64(0.5952380952380952),
  'precision': np.float64(0.423728813559322)},
 'Adam': {'accuracy': 0.5833333333333334,
  'f1_score': np.float64(0.0),
  'precision': np.float64(0.0)},
 'RMSprop': {'accuracy': 0.4166666666666667,
  'f1_score': np.float64(0.5882352941176471),
  'precision': np.float64(0.4166666666666667)}}